## Project: Investigate a Dataset 

### Table of Contents
- <a href='#Introduction'>Introduction</a>

- <a href='#Data Wrangling'>Data Wrangling</a>

- <a href='#Data Cleaning'>Data Cleaning
- <a href='#Exploratory Data Analysis'>Exploratory Data Analysis</a>

- <a href='#Conclusions'>Conclusions</a>

<a id='Introduction'></a>

### Introduction 

######  Project Questions:
- What census data is most associated with high gun per capita? 
- Which states have had the highest growth in gun registrations? 
- What is the overall trend of gun purchases?

###### Data background: FBI Gun Data

The data comes from the FBI's National Instant Criminal Background Check System. The NICS is used by to determine whether a prospective buyer is eligible to buy firearms or explosives. Gun shops call into this system to ensure that each customer does not have a criminal record or isn’t otherwise ineligible to make a purchase. The data has been supplemented with state level data from census.gov.

- The NICS data is found in one sheet of an .xlsx file. It contains the number of firearm checks by month, state, and type.
- The U.S. census data is found in a .csv file. It contains several variables at the state level. Most variables just have one data point per state (2016), but a few have data for more than one year.

In [36]:
# Import packages 
import pandas as pd
import numpy as np
import csv
from datetime import datetime
import matplotlib.pyplot as plt
get_ipython().run_line_magic('matplotlib', 'inline')
% matplotlib inline

<a id='Data Wrangling'></a>

### Data Wrangling

In this section of the report, I will load in the data, check for cleanliness, and then trim and clean the dataset for analysis. 

#### Importing Data

In [37]:
#loading the gun csv file and storing it in the variable "df1"
df1 = pd.read_csv('gun_data.csv')

#loading the census csv file and storing it in the variable "df2"
df2 = pd.read_csv('US_census data.csv')

#### Descriptive Summary of Gun Data

In [38]:
df1.head(2)

,month,state,permit,permit_recheck,handgun,long_gun,other,multiple,admin,prepawn_handgun,...,returned_other,rentals_handgun,rentals_long_gun,private_sale_handgun,private_sale_long_gun,private_sale_other,return_to_seller_handgun,return_to_seller_long_gun,return_to_seller_other,totals
0,2017-09,Alabama,16717.0,0.0,5734.0,6320.0,221.0,317,0.0,15.0,...,0.0,0.0,0.0,9.0,16.0,3.0,0.0,0.0,3.0,32019
1,2017-09,Alaska,209.0,2.0,2320.0,2930.0,219.0,160,0.0,5.0,...,0.0,0.0,0.0,17.0,24.0,1.0,0.0,0.0,0.0,6303


In [39]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12485 entries, 0 to 12484
Data columns (total 27 columns):
month                        12485 non-null object
state                        12485 non-null object
permit                       12461 non-null float64
permit_recheck               1100 non-null float64
handgun                      12465 non-null float64
long_gun                     12466 non-null float64
other                        5500 non-null float64
multiple                     12485 non-null int64
admin                        12462 non-null float64
prepawn_handgun              10542 non-null float64
prepawn_long_gun             10540 non-null float64
prepawn_other                5115 non-null float64
redemption_handgun           10545 non-null float64
redemption_long_gun          10544 non-null float64
redemption_other             5115 non-null float64
returned_handgun             2200 non-null float64
returned_long_gun            2145 non-null float64
returned_other   

In [40]:
print('Number of duplicated rows:')
sum(df1.duplicated())

Number of duplicated rows:


0

In [41]:
#Remove any duplicated rows.
df1.drop_duplicates(keep=False, inplace=True)

#Check if there is any duplicated rows.
sum(df1.duplicated())

0

#### Descriptive Summary of Census Data

In [44]:
#printing first five rows with defined columns of gun_data database
df2.head(2)

,Fact,Fact Note,Alabama,Alaska,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,...,South Dakota,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
0,"Population estimates, July 1, 2016, (V2016)",NaN,"4,863,300","741,894","6,931,071","2,988,248","39,250,017","5,540,545","3,576,452","952,065",...,865454,6651194,"27,862,596","3,051,217","624,594","8,411,808","7,288,000","1,831,102","5,778,708","585,501"
1,"Population estimates base, April 1, 2010, (V2...",NaN,"4,780,131","710,249","6,392,301","2,916,025","37,254,522","5,029,324","3,574,114","897,936",...,814195,6346298,"25,146,100","2,763,888","625,741","8,001,041","6,724,545","1,853,011","5,687,289","563,767"


In [45]:
#Check data type of columns in census data 
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85 entries, 0 to 84
Data columns (total 52 columns):
Fact              80 non-null object
Fact Note         28 non-null object
Alabama           65 non-null object
Alaska            65 non-null object
Arizona           65 non-null object
Arkansas          65 non-null object
California        65 non-null object
Colorado          65 non-null object
Connecticut       65 non-null object
Delaware          65 non-null object
Florida           65 non-null object
Georgia           65 non-null object
Hawaii            65 non-null object
Idaho             65 non-null object
Illinois          65 non-null object
Indiana           65 non-null object
Iowa              65 non-null object
Kansas            65 non-null object
Kentucky          65 non-null object
Louisiana         65 non-null object
Maine             65 non-null object
Maryland          65 non-null object
Massachusetts     65 non-null object
Michigan          65 non-null object
Minnesota

In [46]:
print('Number of duplicated rows:')
sum(df2.duplicated())

Number of duplicated rows:


3

In [47]:
#Remove any duplicated rows.
df2.drop_duplicates(keep=False, inplace=True)

#Check if there is any duplicated rows.
sum(df2.duplicated())

0

<a id='Data Cleaning'></a>
### Data Cleaning

In [48]:
#Check the Fact Note's number of NaN values 
df2['Fact Note'].isnull().sum().sum() 

53

In [49]:
#Replace the Nan field with mean of each column for gun data  
df1.fillna(df1.mean(), axis=0, inplace=True)

In [50]:
print('Sum of null values:')
df1.isnull().sum().sum()

Sum of null values:


0

In [51]:
#Drop fact Note column sadince it is not used and will affect groupby function.
df2.drop(['Fact Note'],axis=1)

,Fact,Alabama,Alaska,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Florida,...,South Dakota,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
0,"Population estimates, July 1, 2016, (V2016)","4,863,300","741,894","6,931,071","2,988,248","39,250,017","5,540,545","3,576,452","952,065","20,612,439",...,865454,6651194,"27,862,596","3,051,217","624,594","8,411,808","7,288,000","1,831,102","5,778,708","585,501"
1,"Population estimates base, April 1, 2010, (V2...","4,780,131","710,249","6,392,301","2,916,025","37,254,522","5,029,324","3,574,114","897,936","18,804,592",...,814195,6346298,"25,146,100","2,763,888","625,741","8,001,041","6,724,545","1,853,011","5,687,289","563,767"
2,"Population, percent change - April 1, 2010 (es...",1.70%,4.50%,8.40%,2.50%,5.40%,10.20%,0.10%,6.00%,9.60%,...,0.063,0.048,10.80%,10.40%,-0.20%,5.10%,8.40%,-1.20%,1.60%,3.90%
3,"Population, Census, April 1, 2010","4,779,736","710,231","6,392,017","2,915,918","37,253,956","5,029,196","3,574,097","897,934","18,801,310",...,814180,6346105,"25,145,561","2,763,885","625,741","8,001,024","6,724,540","1,852,994","5,686,986","563,626"
4,"Persons under 5 years, percent, July 1, 2016, ...",6.00%,7.30%,6.30%,6.40%,6.30%,6.10%,5.20%,5.80%,5.50%,...,0.071,0.061,7.20%,8.30%,4.90%,6.10%,6.20%,5.50%,5.80%,6.50%
5,"Persons under 5 years, percent, April 1, 2010",6.40%,7.60%,7.10%,6.80%,6.80%,6.80%,5.70%,6.20%,5.70%,...,0.073,0.064,7.70%,9.50%,5.10%,6.40%,6.50%,5.60%,6.30%,7.10%
6,"Persons under 18 years, percent, July 1, 2016,...",22.60%,25.20%,23.50%,23.60%,23.20%,22.80%,21.10%,21.50%,20.10%,...,0.246,0.226,26.20%,30.20%,19.00%,22.20%,22.40%,20.50%,22.30%,23.70%
7,"Persons under 18 years, percent, April 1, 2010",23.70%,26.40%,25.50%,24.40%,25.00%,24.40%,22.90%,22.90%,21.30%,...,0.249,0.236,27.30%,31.50%,20.70%,23.20%,23.50%,20.90%,23.60%,24.00%
8,"Persons 65 years and over, percent, July 1, 2...",16.10%,10.40%,16.90%,16.30%,13.60%,13.40%,16.10%,17.50%,19.90%,...,0.16,0.157,12.00%,10.50%,18.10%,14.60%,14.80%,18.80%,16.10%,15.00%
9,"Persons 65 years and over, percent, April 1, 2010",13.80%,7.70%,13.80%,14.40%,11.40%,10.90%,14.20%,14.40%,17.30%,...,0.143,0.134,10.30%,9.00%,14.60%,12.20%,12.30%,16.00%,13.70%,12.40%


In [52]:
df2.isnull().sum().sum()

854

In [53]:
#Convert data type from string to float for all the state column in Census data
#Assign state to a list 
states=['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia',  
        'Hawaii', 'Idaho', 'Illinois','Indiana', 'Iowa', 'Kansas', 
        'Kentucky', 'Louisiana', 'Maine','Maryland', 'Massachusetts', 
        'Michigan','Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska',
       'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
       'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon','Pennsylvania', 
        'Rhode Island', 'South Carolina','South Dakota', 'Tennessee', 'Texas', 'Utah', 
        'Vermont','Virginia', 'Washington', 'West Virginia','Wisconsin', 'Wyoming']

In [54]:
# For the column name in states list, remove all the non digit character and convert it to float
for state in states:
    df2[state].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
    # remove all the non digit character
    df2[state]=pd.to_numeric(df2[state], downcast='float', errors='ignore')
    #convert data type to float and ignore the nan value
    #Using census.dtypes to check result, The result shows all the state columns's data type now is float32. 


In [55]:
#Step 4: Extract Year and months column based on Month in Gun data
# and convert data type from string to numeric 
#Assign the data before '-' in month column to year column 
df1['year']=df1['month'].apply(lambda x: x.split("-")[0]).astype(int)
df1['year'].unique()

array([2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009, 2008, 2007,
       2006, 2005, 2004, 2003, 2002, 2001, 2000, 1999, 1998])

In [56]:
#Assign the data after '-' in month column to year column
df1['months'] = df1['month'].apply(lambda x: x.split("-")[1]).astype(int)
df1['months'].unique()


array([ 9,  8,  7,  6,  5,  4,  3,  2,  1, 12, 11, 10])

In [57]:
df1.head(1) # Check year and months columns are added in the gun data


,month,state,permit,permit_recheck,handgun,long_gun,other,multiple,admin,prepawn_handgun,...,rentals_long_gun,private_sale_handgun,private_sale_long_gun,private_sale_other,return_to_seller_handgun,return_to_seller_long_gun,return_to_seller_other,totals,year,months
0,2017-09,Alabama,16717.0,0.0,5734.0,6320.0,221.0,317,0.0,15.0,...,0.0,9.0,16.0,3.0,0.0,0.0,3.0,32019,2017,9


In [58]:
#Change all float columns to int since the count of gun should be integer 
#Assign columns name  to a list 
cols = ['permit', 'permit_recheck', 'handgun','long_gun','other','admin','prepawn_handgun','prepawn_long_gun',            
'prepawn_other','redemption_handgun', 'redemption_long_gun','redemption_other','returned_handgun','returned_long_gun' ,          
'returned_other','rentals_handgun', 'rentals_long_gun', 'private_sale_handgun' , 'private_sale_long_gun',        
'private_sale_other', 'return_to_seller_handgun',  'return_to_seller_long_gun','return_to_seller_other']

In [59]:
#Convert the column name in cols list into int64 format using applymap
df1[cols] = df1[cols].applymap(np.int64)

<a id='Exploratory Data Analysis'></a>
### Exploratory Data Analysis

Moving on to exploration. We compute statistics and create visualizations with the goal of addressing the research questions that are posed in the Introduction section. 

#### Research Question 1: What census data is most associated with high gun per capita?

In [60]:
#Transpose Census data and remove the index on Fact
df2.set_index('Fact',inplace=True)
df2_T = df2.T.reset_index()

In [61]:
#Rename the column name from index to state to match the column name in gun data. 
df2_T.rename(columns={'index':'state'},inplace = True)

In [62]:
#Get subset data for 2016 and 2010
df1_16=df1[df1['year'] == 2016]
df1_10=df1[df1['year'] == 2010]

In [63]:
#Group by gun data by state and gun totals for 2010 and 2016 
df1total_16= df1_16.groupby(['state'])['totals'].sum().reset_index()
df1total_10= df1_10.groupby(['state'])['totals'].sum().reset_index()

In [64]:
#Rename the dataset to represent different time point 
df1total_16.rename(columns={'totals':'2016_totals'},inplace = True)
df1total_10.rename(columns={'totals':'2010_totals'},inplace = True)

In [65]:
#Merge 2010 and 2016 gun data summary 
df1total = df1total_16.merge(df1total_10, on='state', how='inner')

In [66]:
#Merge gun and census data with inner join, by state column 
result = df1total.merge(df2_T, on='state', how='inner')

In [67]:
#calculate Gun_Per_Capital for 2016
result['Gun_Per_Capital_2016'] = result['2016_totals']/result['Population estimates, July 1, 2016,  (V2016)']

In [68]:
#calculate Gun_Per_Capital for 2010
result['Gun_Per_Capital_2010'] = result['2010_totals']/result['Population estimates base, April 1, 2010,  (V2016)']

In [71]:
#Top 5  the highest state per capital on 2010
result.nlargest(5,'Gun_Per_Capital_2010')

TypeError: Column 'Gun_Per_Capital_2010' has dtype object, cannot use method 'nlargest' with this dtype

In [ ]:
#Top 5  the highest gun per capital state on 2016
result.nlargest(5,'Gun_Per_Capital_2016')

In [ ]:
#Drop non fact value
fact = result.drop(['Gun_Per_Capital_2010','state','FIPS Code','2016_totals','2010_totals'],axis=1)

In [ ]:
#create scatter plot for all the fact variable in speparate figure by Name Group
for col in list(fact):
    if 'firms' in col:
        plt.scatter(fact[col],fact['Gun_Per_Capital_2016'], label =col)
           
plt.ylabel("Gun_Per_Capital_2016")
plt.title("figure1: firms Variable and Gun_Per_Capital Scatter Plot")
plt.grid(True)
plt.legend(bbox_to_anchor=(1.1, 1.05))
plt.show()

In [ ]:
#figure2: Employment related Variable and Gun_Per_Capital 
#Use for loop to create scatter plot for Employment variable in one figure 
for col in list(fact):
    if 'employ' in col:
        plt.scatter(fact[col],fact['Gun_Per_Capital_2016'], label =col)
        plt.xlabel(col)
plt.ylabel("Gun_Per_Capital_2016")
plt.title("Figure4: Employee Variable and Gun_Per_Capital Scatter Plot")
plt.grid(True)
plt.legend(bbox_to_anchor=(1.1, 1.05))
plt.show()

In [ ]:
#Keep only 6 variable in fact table to create scatter plot with Gun Per capital since they have week association
imp =['White alone, percent, July 1, 2016,  (V2016)',
    'Persons 65 years and over, percent, April 1, 2010',
    'Owner-occupied housing unit rate, 2011-2015',
    'Asian alone, percent, July 1, 2016,  (V2016)',
    'Foreign born persons, percent, 2011-2015', 
    'Median gross rent, 2011-2015']

In [ ]:
#create scatter plot for all the fact variable in speparate figure, 6 figures

for col in imp:
    plt.figure(figsize=(4,4))
    print(col)
    plt.scatter(fact[col],fact['Gun_Per_Capital_2016'], label =col)
    plt.title(col+" and Gun_Per_Capital Scatter Plot")     
    plt.ylabel("Gun_Per_Capital_2016")
    plt.xlabel(col)
    plt.grid(True)
    plt.show()

#### Research Question 2: Which states have had the highest growth in gun registrations?

In [ ]:
#Calculate the increasing percentage of gun registrations from 2010 to 2016
result['gun_growth'] = result['2016_totals']/(result['2016_totals']-result['2010_totals'])

In [ ]:
#Get the biggest growth percentage 
result['gun_growth'].max()*100 # the result of the biggest growth perentage  is 403.20%

In [ ]:
#List Top 5 rows by gun growth rate descending
result.nlargest(5,'gun_growth')

In [ ]:
#Create Bar chart for every states'gun growth 
#Set the figure size 
plt.figure(figsize=(30,30))

plt.rcdefaults()
fig, ax = plt.subplots()

#Sort result data by gun_growth value
sorted = result.sort_values(by=['gun_growth'])

#create bar chart 
y_pos = np.arange(len(sorted['state']))
error = np.random.rand(len(sorted['state']))
ax.barh(y_pos, (sorted['gun_growth']*100), xerr=error, align='center',height=2,linewidth=5,color='green', ecolor='black')

#set x and y axis lable and make the label readable
ax.set_yticks(y_pos)
ax.set_xlabel("Gun Registration Growth %")
ax.set_yticklabels(sorted['state'],size=6)

#Invert x and y axis
ax.invert_yaxis()  # labels read top-to-bottom

#Set tick colors:
ax.tick_params(axis='x', colors='green')
ax.tick_params(axis='y', colors='black')

#Set the title
plt.title("State Gun Registration Growth from 2010 and 2016")     
plt.grid(True)
plt.legend(bbox_to_anchor=(1.1, 1.05))
plt.show()

#### Research Question 3: What is the overall trend of gun purchases?

In [ ]:
#Assign figure size
plt.figure(figsize=(10,5))

df1.groupby('month')['totals'].sum().plot(kind='line',sharex=True, sharey=True, layout = (2, 1))
#set x and y axis lable name
plt.xlabel('Month')
plt.ylabel('Total gun purchases')
plt.legend()
plt.title("Gun Purchase Trend by months Line Chart")
plt.show()

#2. Create line chart using grouped data by year, which can be used to observe the change during the season
plt.figure(figsize=(10,6))
df1.groupby('year')['totals'].sum().plot(kind='line')

plt.ylabel('Total gun purchases')
plt.xlabel('year')
plt.title("Gun Purchase Trend by years Line Chart")
plt.legend()
plt.show()


<a id='Conclusions'></a>
### Conclusions

Tip: Finally, summarize your findings and the results that have been performed. Make sure that you are clear with regards to the limitations of your exploration. If you haven't done any statistical tests, do not imply any statistical conclusions. And make sure you avoid implying causation from correlation!

Tip: Once you are satisfied with your work here, check over your report to make sure that it is satisfies all the areas of the rubric (found on the project submission page at the end of the lesson). You should also probably remove all of the "Tips" like this one so that the presentation is as polished as possible.

### Submitting your Project

Before you submit your project, you need to create a .html or .pdf version of this notebook in the workspace here. To do that, run the code cell below. If it worked correctly, you should get a return code of 0, and you should see the generated .html file in the workspace directory (click on the orange Jupyter icon in the upper left).

Alternatively, you can download this report as .html via the File > Download as submenu, and then manually upload it into the workspace directory by clicking on the orange Jupyter icon in the upper left, then using the Upload button.

Once you've done this, you can submit your project by clicking on the "Submit Project" button in the lower right here. This will create and submit a zip file with this .ipynb doc and the .html or .pdf version you created. Congratulations!

In [ ]:
from subprocess import call
call(['python', '-m', 'nbconvert', 'Investigate_a_Dataset.ipynb'])